In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from collections import Counter
from konlpy.tag import Okt
import re
import os

okt=Okt()

data = pd.read_csv('./data/total_reault.csv') #원본 데이터
#tf불러오기
with open('./data/titleTF.pickle', 'rb') as f:
    tf = pickle.load(f)
#idf불러오기
with open('./data/titleIDF.pickle', 'rb') as f:
    idf = pickle.load(f)
#tfidf불러오기
with open('./data/titleTFIDF.pickle', 'rb') as f:
    tfidf = pickle.load(f)
    

In [18]:
class Similarity():
   
    def __init__(self,new_text):
        self.new_text=new_text
        
    def get_new_text_tfdtm(self):
        # 새로운 글 명사 추출 후 TF구하기
        self.new_nouns=okt.nouns(self.new_text) # new_text 명사추출
        self.new_tf=pd.Series(Counter(self.new_nouns)) # new_text TF구하기
        self.new_text_filter=[i for i in self.new_tf.index if i in idf.index] # 새로 생긴 단어(idf에 없는 단어) 없애기
        self.new_tf=self.new_tf[self.new_text_filter]
        return self.new_tf
    
    def get_new_text_tfidf(self):
        self.new_tfidf=self.new_tf*idf
        self.new_tfidf=self.new_tfidf.fillna(0)
        self.new_tfidf=self.new_tfidf.values.reshape(1,-1)
        return self.new_tfidf
    
    def get_similarity3(self):
        self.get_new_text_tfdtm()
        self.get_new_text_tfidf()
        # 새로운 글과 가장 유사한 원본데이터(data)의 행 찾기
        self.sim_res=[cosine_similarity(self.new_tfidf,tfidf.loc[i].values.reshape(1,-1)).flatten().tolist() for i in tfidf.index]
        self.sim_res_dict=[(idx,i[0]) for idx, i in enumerate(self.sim_res)]
  
        # 유사한 글 30개 추출(유사도 내림차순으로 정렬)
        self.sim30_sort=sorted(self.sim_res_dict, key=lambda x: x[1], reverse=True)[:30] # 유사한 문서 30개 (행번호,코사인유사도 값)
        self.sim30_sort_index=[i[0] for i in self.sim30_sort] #  유사한 행번호 30개, 내림차순 정렬
        self.sim30_sort_score=[i[1] for i in self.sim30_sort] #  유사도 점수 30개, 내림차순 정렬
        #저장
        filename=' '.join(re.findall('[가-힣0-9a-zA-Z]+',self.new_text)) #파일이름(한글,숫자, 영어)로 제한, 특수문자 제외
        data.iloc[self.sim30_sort_index].to_csv(f'./data/제목_similarity30_{filename}.csv')
        print(f'"{os.getcwd()}"폴더에 유사한 글 30개가 "./data/제목_similarity30_{filename}.csv" 파일명으로 저장되었습니다.')  
        return data.iloc[self.sim30_sort_index]

    def view_similarity3(self):
        
        # 새로운글과 유사한 문장 3개 보여주기
        self.df=self.get_similarity3()
        print('==============================================================================================')
        print(f'   >>> 질문(Q) : {self.new_text}')
        print('==============================================================================================')
        
        # -----<수정>  질문에 대한 응답 여러개 처리
        CNT=0
        TMP=None
        self.result=[]
        tmp=''

        # 질문에 해당하는 답변 
        for idx,i in enumerate(self.df.index):
            if CNT>3:
                break
            if self.df['title'][i]!=TMP:
                self.result.append(tmp)
                CNT+=1
                유사도=self.sim30_sort_score[idx]
                날짜=self.df['날짜'][i]
                제목=self.df['title'][i]
                질문=self.df['questions'][i]
                응답=self.df['answer'][i]
                tmp={
                    'CNT':CNT,
                    '유사도':유사도,
                    '날짜':날짜,
                    '제목':제목,
                    '질문':질문,
                    '응답':[응답]
                }
                TMP=self.df['title'][i]
            else:
                응답=self.df['answer'][i]
                tmp['응답'].append(응답)
        
        # 유사한 질문 3가지 출력
        for i in self.result[1:]:
            #print(f'{i['CNT']}')
            print('[{}] 유사도:{}'.format(i['CNT'], i['유사도']))
            print()
            print('   ▶ 제목: {}'.format(i['제목']))
            print()
            print('   ▶ 질문')
            print(i['질문'])
            print()
            print('   ▶ 응답')
            for idx, j in enumerate(i['응답'],1):
                print(f'응답{idx}: {j}')
                print()
            print('--------------------------------------------------------------------------------------')
            
if __name__=='__main__':
    Q='전세보증금 못 받고 있어요'
    s_text=Similarity(Q)
    s_text.view_similarity3()


"C:\Users\hmh\Documents\py\_프로젝트\최종_문서 유사도\최종문서유사도 - 1"폴더에 유사한 글 30개가 "./data/제목_similarity30_전세보증금 못 받고 있어요.csv" 파일명으로 저장되었습니다.
   >>> 질문(Q) : 전세보증금 못 받고 있어요
[1] 유사도:1.0000000000000002

   ▶ 제목: 전세보증금 못 받고 있어요

   ▶ 질문
계약기간 종료 3개월전 메세지 전달.내용증명 2차례 발송하였으나 반송.현재 계약기간 종료. 아파트 보증금 칠천만원 못 받고 있음. 집주인은 칠천만원 매매로 집을 내놓았고 그돈으로 전세금을 주겠다고 함.현재 이사예정으로 임대차등기명령 신청 후 송달 중임.참고로 집주인은 임대업자이고 깡통전세인것 같은데1. 전세금 안 받고 아파트를 넘겨 받았는데 그 방법이 좋은 건가요?집을 경매로 넘기는 것보다 나은 방법인가요?저는 이 아파트를 사고 싶지 않아서요.외곽에 있고 오래된 아파트라 매매가 안될 것 같아서요2.소송을 생각 중인데 등기완료 후 지급명령 신청을 해야하나요? 소송 순서 좀 알려주세요. 가압류신청도 하는 것 같은데 그런 순서를 알고 싶어서요3. 혹시 집이 경매로 넘어가서 제가 그집을 낙찰받게 되면 제가 돈을 내야 하는게 있나요?얼핏 듣기로 상계처리랑 세금안낸것을 제가 내야 된다는 이야기도 있고 그렇던데요.

   ▶ 응답
응답1: 금액이 매우 크시네요.​더 이상을 시간 지체 하지마시고, 소송 하시는걸 추천해드립니다.​요즘 이런 피해사례가 많아 지고 있습니다. ​민/형사상 가능한 모든 방법을 동원해 도와드리고 있습니다.​대형로펌이며, 연락 주시면 친절히 상담 해드리겠습니다.​카톡 : hooni89

응답2: 적은 금액이 아니어서 몇달간 마음고생이 많으셨겠습니다지금 진행된 상황이 아파트를 이미 넘겨 받으신건가요?질문주신 세가지 사항은 구체적인 내용을 짚어가며 답변드릴 수 있을 것 같습니다카톡 born2crazy로 문의주시면 바로 도움드리겠습니다

응답3: ​​ 합의가 안되면,민사 